# Streaming Results

Here is an example pattern if you want to stream your multiple results. Note that this is not supported for Hugging Face text completions at this time.


In [ ]:
!python -m pip install semantic-kernel==0.9.0.beta1

In [ ]:
from services import Service

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = Service.AzureOpenAI

In [ ]:
import semantic_kernel as sk
from semantic_kernel.models.ai.chat_completion.chat_history import ChatHistory

if selectedService == Service.OpenAI or selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.open_ai_prompt_execution_settings import (
        OpenAITextPromptExecutionSettings,
        OpenAIChatPromptExecutionSettings,
    )
    from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
        AzureChatPromptExecutionSettings,
    )
    from semantic_kernel.connectors.ai.open_ai import (
        AzureTextCompletion,
        AzureChatCompletion,
        OpenAITextCompletion,
        OpenAIChatCompletion,
    )
if selectedService == Service.HuggingFace:
    from semantic_kernel.connectors.ai.hugging_face import HuggingFaceTextCompletion

First, we will set up the text and chat services we will be submitting prompts to.


In [ ]:
kernel = sk.Kernel()

# Configure Azure LLM service
if selectedService == Service.AzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    azure_text_service = AzureTextCompletion(
        service_id="aoai_text", deployment_name="text-davinci-003", endpoint=endpoint, api_key=api_key
    )  # set the deployment name to the value of your text model (e.g. gpt-35-turbo-instruct or text-davinci-003)
    azure_chat_service = AzureChatCompletion(
        service_id="aoai_chat", deployment_name="gpt-35-turbo", endpoint=endpoint, api_key=api_key
    )  # set the deployment name to the value of your chat model

# Configure OpenAI service
if selectedService == Service.OpenAI:
    api_key, org_id = sk.openai_settings_from_dot_env()
    oai_text_service = OpenAITextCompletion(
        service_id="oai_text", ai_model_id="gpt-3.5-turbo-instruct", api_key=api_key, org_id=org_id
    )
    oai_chat_service = OpenAIChatCompletion(
        service_id="oai_chat", ai_model_id="gpt-3.5-turbo", api_key=api_key, org_id=org_id
    )

# Configure Hugging Face service
if selectedService == Service.HuggingFace:
    hf_text_service = HuggingFaceTextCompletion(ai_model_id="distilgpt2", task="text-generation")

Next, we'll set up the completion request settings for text completion services.


In [ ]:
oai_prompt_execution_settings = OpenAITextPromptExecutionSettings(
    service_id="oai_text",
    max_tokens=150,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
)

## Streaming Open AI Text Completion


In [ ]:
if selectedService == Service.OpenAI:
    chat = ChatHistory()
    chat.add_user_message("What is the purpose of a rubber duck?")
    stream = oai_text_service.complete_stream(chat_history=chat, settings=oai_prompt_execution_settings)
    async for message in stream:
        print(str(message[0]), end="")  # end = "" to avoid newlines

## Streaming Azure Open AI Text Completion


In [ ]:
if selectedService == Service.AzureOpenAI:
    chat = ChatHistory()
    chat.add_user_message("provide me a list of possible meanings for the acronym 'ORLD'")
    stream = azure_text_service.complete_stream(chat_history=chat, settings=oai_prompt_execution_settings)
    async for message in stream:
        print(str(message[0]), end="")

## Streaming Hugging Face Text Completion


In [ ]:
if selectedService == Service.HuggingFace:
    from semantic_kernel.connectors.ai.hugging_face.hf_prompt_execution_settings import (
        HuggingFacePromptExecutionSettings,
    )

    hf_prompt_execution_settings = HuggingFacePromptExecutionSettings(
        service_id="hf_text",
        extension_data={
            "max_new_tokens": 80,
            "top_p": 1,
            "eos_token_id": 11,
            "pad_token_id": 0,
        },
    )

In [ ]:
if selectedService == Service.HuggingFace:
    chat = ChatHistory()
    chat.add_user_message("The purpose of a rubber duck is")
    stream = hf_text_service.complete_stream(chat_history=chat, prompt_execution_settings=hf_prompt_execution_settings)
    async for text in stream:
        print(str(text[0]), end="")  # end = "" to avoid newlines

Here, we're setting up the settings for Chat completions.


In [ ]:
oai_chat_prompt_execution_settings = OpenAIChatPromptExecutionSettings(
    service_id="oai_chat",
    max_tokens=150,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
)

## Streaming OpenAI Chat Completion


In [ ]:
if selectedService == Service.OpenAI:
    content = "You are an AI assistant that helps people find information."
    chat = ChatHistory()
    chat.add_system_message(content)
    stream = oai_chat_service.complete_chat_stream(chat_history=chat, settings=oai_chat_prompt_execution_settings)
    async for text in stream:
        print(str(text[0]), end="")  # end = "" to avoid newlines

## Streaming Azure OpenAI Chat Completion


In [ ]:
az_oai_chat_prompt_execution_settings = AzureChatPromptExecutionSettings(
    service_id="aoai_chat",
    max_tokens=150,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
)

In [ ]:
if selectedService == Service.AzureOpenAI:
    content = "You are an AI assistant that helps people find information."
    chat = ChatHistory()
    chat.add_system_message(content)
    chat.add_user_message("What is the purpose of a rubber duck?")
    stream = azure_chat_service.complete_chat_stream(chat_history=chat, settings=az_oai_chat_prompt_execution_settings)
    async for text in stream:
        print(str(text[0]), end="")  # end = "" to avoid newlines